<a href="https://colab.research.google.com/github/OksanaTymchenko/BudgetCheck/blob/master/BudgetAmendments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install -q docx2python
import numpy as np
import pandas as pd
import re
# from docx2python import docx2python
from functools import reduce
import openpyxl 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# reading from source doc file

budget_doc = docx2python('/content/drive/My Drive/Colab Notebooks/budget.docx')
budget = list(filter(lambda x: len(x) > 1, budget_doc.body))
budget = budget[1:9]

def clean_parts(raw_budget):
    temp = pd.DataFrame(raw_budget[1:])
    temp.drop(temp.columns[:-2], axis=1, inplace=True)
    temp.dropna(inplace=True)
    temp.columns = ['Правки', 'Висновки']
    # temp.dropna(subset = ['Правки'], inplace=True)
    temp = temp[temp.iloc[:,0].map(lambda d: len(d[0]) > 0)]
    print(temp.head())
    return temp

budget_df = list(map(clean_parts, budget))
budget_df = pd.concat(budget_df, ignore_index = True)

def edit_split(edit):
    if edit[0] and edit[0].startswith('-'):
      number, first_author = edit[0].split(maxsplit = 1)
      author_list = [first_author]
      k = 1
      while 'Н.д.' in edit[k]:
        author_list.append(edit[k])
        k += 1
      text = '\n'.join(edit[len(author_list):])
      return number, author_list, text
    else: 
      return None

In [0]:
# reading from table copied from doc

edits_df = pd.read_excel('/content/drive/My Drive/budget_table.xlsx')

def split_edit(edit):
  if edit.startswith('-'):
    number, rest = edit.split(maxsplit = 1)
    first_author, rest = rest.split('\n', maxsplit = 1)
    author_list = [first_author]
    while 'Н.д.' in rest:
      deputy, rest = rest.split('\n', maxsplit = 1)
      author_list.append(deputy)
    return number, author_list, rest
  else: 
    return None

In [0]:
def get_edit_number(number):
  return re.search('\d+', number).group()

In [0]:
# a, b, c = edit_split(budget_df.loc[439]['Правки'])
def get_deputies(dep_list):
  try:
    deputies = []
    grp = False
    if re.match('\s*(Комітет|Фракція)', dep_list[0]):
      grp = True
      deputies.append({'deputy':dep_list[0], 'card':None})
      dep_list = dep_list[1:]
    for dep in dep_list:
      res = re.search(r"Н.д.([^(]+).*№(\d+)", dep)
      if res:
       deputies.append({'deputy':res.group(1).strip(), 'card':res.group(2)})
      else:
        res = re.search(r".([^(]+).*№(\d+)", dep)
        deputies.append({'deputy':res.group(1).strip(), 'card':res.group(2)})
    if len(deputies) > 1:
      grp = True
    return deputies, grp
  except:
    print('depError', dep_list)


In [0]:
def get_action(edit):
  # try:
    keywords = ['(збільшити|передбачити)(.*?)(видатк(и|ах)|субвенці(ю|ї)|дотаці(ю|ї))',
                # '((збільши(ти|вши))|передбачити|встановити).*(видатк(и|ів)|субвенці(ю|ї)|дотаці(ю|ї)|обсяг|кошт(и|ів))',
                # -зменшивши!!!
                # '(встановити) .*збільшивши',
                '(включити|відновити|передбачити)(.*?)бюджетну програму',
                '(встановити)(.*?)(видатки|субвенці(ю|ї))',
                '(збільшити)(.*?)обсяг (асигнувань|видатків)',
                '(встановити|(від|по)новити)(.*?)субвенці(ю|ї)',
                '(доповнити)(.*?)програм(ою|ами)',
                '(збільшити|відновити) фінансування', '(включити) (((в|у) роз)|до)', 
                '(Передбачити) нову програму',
                '(передбачити|виділити)(.*?)[^«"]кошти',
                '(надати) додатково',
                '(визначити) видатки',
                '^(?:видатки|субвенцію)(?:.*?)(збільшити)',
                '^(визначити) обсяг',
                'видатки (встановити)',
                '(передбачити) збільшення(.*?)видатків',
                # '(встановити) .*тобто збільшити', 
                '(збільшення) видаткової частини', 
                # '(?:видатк(?:и|ів)|субвенці(?:ю|ї)|дотаці(?:ю|ї)).*(збільшити|передбачити|визначити|встановити)', 
                # '^встановити видатки'
                '(передбачити) \d+',
                'субвенцію (визначити)',
                '(передбачається) сума',
                '(передбачити) програму',
                '(спрямовується) за рахунок',
                'дотацію(?:.*?)(збільшити)',
                '(забезпечити) (встановлення обсягів видатків|додаткове фінансування)',
                'впровадження програми(?:.*?)(збільшити)'
                ]
    action_list = []

    for keyword in keywords:
      sentence = '[^\.]*'+ keyword
      act = re.compile(sentence, re.IGNORECASE)
      res = act.search(edit)
      if res:
        edit = edit[res.start():]
        if re.search('\d+', edit) and not re.search('(тобто)? зменши(ти|вши)', edit):
          return res

    return None


In [0]:
def get_sum_changing(edit):
  money_str = '«?([\d.,]+)»?\s?(тис|млн|млрд|тисяч)?\.?\s?(грн|грив(ень|ні|ня))?'
  changing_str = '(цифри|суму) '+ money_str + '\s?замінити(.*?)'+ money_str
  change = re.findall(changing_str, edit)
  res = 0
  for c in change:
    from_sum = format_sum(c[1], 'тис')
    to_sum = format_sum(c[6], 'тис')
    if to_sum > from_sum:
      res += to_sum - from_sum
  return res
      # return to_sum - from_sum


In [0]:
def format_sum(amount_str, order_str):
  # order = money_match[1].strip(' .')
  try:
    amount_str = float(amount_str.replace(' ','').replace('.', '').replace(',','.'))
    switcher = {
        'тис': amount_str*10**3,
        'млн': amount_str*10**6,
        'млрд': amount_str*10**9,
        'тисяч': amount_str*10**3,
        'млр': amount_str*10**9
    }
    return switcher.get(order_str, amount_str)
  except:
    return 0


In [0]:
def get_code(edit):

    code_1 ='(за кодом|КПКВК|програм(ою|у|и)|по коду|код|за кодами) (?P<code>\d+)'
    code_2 = '\((?P<code1>\d+)\)'
    code = re.compile(code_1+'|'+code_2, re.IGNORECASE)
    res = code.search(edit)
    if res:
      if res.group('code'):
        return res.group('code')
      elif res.group('code1'):
        return res.group('code1')
   
    return None

In [0]:
def get_percent(edit):
    # percent = re.search('[^\.\n]*[\d\.,]+\s*(%|відсотк(ів|ка))[^\.\n]*', edit)
    percent = re.search('([\d\.,]+)\s*(%|відсотк(ів|ка)) (?P<gdp>ВВП|валового)?[^\.\n]*', edit)
    # percent = re.search(' [\d\.\,]+\s*(%|відсотків|відсотка) ', edit)
    
    return percent


In [0]:
def get_level(edit):
  level = re.search('на рівні (.*?)[\d.,%?]+[^\.\n]*', edit)
  if level:
    return level.group(0)
  else:
    return None
  

In [0]:
def get_at_expense(edit):
  exp = re.compile('(за рахунок|джерелом|джерелами|(шляхом.*зменшення)).*\.', re.DOTALL | re.IGNORECASE)
  exp = exp.search(edit)
  compensated = False
  if exp:
    if re.search('зменш(ення|ити)', exp.group()):
      compensated = True 
    return exp, compensated
  else:
    return None


In [0]:
def find_price(edit):
  money_str = '(?P<amount>[\d., ]+)(?P<order>тис|млн|млрд|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
  # includes_regex = re.compile(money_str + '.*((у|в) тому числі|(у|в) т\.\s?ч\.|з (них|яких)).*' + money_str, re.DOTALL)
  # s_prep = '(на (суму)?|(в|у) обсязі|(в|у) сумі|(в|у) розмірі|до|обсягом|на рівні|не менше)'
  increase_str = '(збільш(ити|ивши))(.*?)на (суму )?' + money_str
  price = re.search(increase_str, edit, re.IGNORECASE)
  if price:
    return format_sum(price.group('amount'), price.group('order'))
  else:
    hrn = '(.*?)(грн|грив(ень|ні|ня))(.*?)'
    increase_str = '(збільшити)(.*?) з ' + hrn + ' до ' + hrn
    price = re.search(increase_str, edit, re.IGNORECASE)
    if price:
      # print(price.group())
      from_str, to_str = price.group().split(' до ', maxsplit=1)
      from_str = re.search(money_str, from_str)
      to_str = re.search(money_str, to_str)
      from_number = format_sum(from_str.group('amount'), from_str.group('order'))
      to_number = format_sum(to_str.group('amount'), to_str.group('order'))
      return to_number - from_number
  return None

In [0]:
def get_sums(edit):
    # money_str = '([\d\.,]+)\s?([а-я]{,5})?\.?\s?(грн|грив(ень|ні|ня))'
    money_str = '([\d., ]+)(тис|млн|млрд|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
    # money_str = '([\d.,]+)\s?(тис|млн|млрд?|тисяч)?\.?\s?(грн|грив(ень|ні|ня))'
    money_sums = re.findall(money_str, edit)
    if money_sums and len(money_sums) == 1:
      return format_sum(money_sums[0][0], money_sums[0][1])
    elif len(money_sums) > 1:
      # brackets_regex = re.compile(money_str + '[^\(]*\([^\)]*' + money_str + '[^\)\(]*\)')
      brackets_regex = re.compile(money_str + '[^\(]*\([^\)]*' + money_str)
      if brackets_regex.search(edit):
        return format_sum(money_sums[0][0], money_sums[0][1])
      else:
        includes_regex = re.compile(money_str + 
                                    '.*((у|в) тому числі|(у|в) т\.\s?ч\.|з (них|яких)|зокрема|одночасно|а саме|В межах|:).*' 
                                    + money_str, re.DOTALL)
        if includes_regex.search(edit):
          return format_sum(money_sums[0][0], money_sums[0][1])
        else:
          first = format_sum(money_sums[0][0], money_sums[0][1])
          rest = sum([ format_sum(s[0], s[1]) for s in money_sums[1:]])
          if first == rest:
            return first
          else: 
            return first + rest
    return None


In [0]:
def edit_to_dict(raw_number, deputies_list, text, action, price):
  GDP = 4551.7 * 10**9
  if type(action) is str:
    start = end = 0
  else:
    start = action.start()
    end = action.end()
    action = action.group(1)
  at_exp = get_at_expense(text[end:])
  compensated = False
  end_lookup = len(text)
  if at_exp:
    end_lookup = end + at_exp[0].start()
    # edit_part = text[action_start:action_end + exp_start]
    compensated = at_exp[1]
    at_exp = at_exp[0].group()
  edit_part = text[start:end_lookup]
  level = get_level(edit_part)
  if not price:
    price = find_price(edit_part)
  if price:
    money = price
  else:
    money = get_sums(edit_part)

  percent_match = get_percent(edit_part)
  if percent_match:
    percent = percent_match.group()
    if percent_match.group('gdp'):
      percent_number = float(percent_match.group(1).replace(' ','').replace(',','.'))
      money = percent_number*GDP/100
  else: 
    percent = percent_match
  if money or percent or level:
    number = get_edit_number(raw_number)
    deputies, group = get_deputies(deputies_list)
    code = get_code(edit_part)
    edit_dict = {'Номер правки':number, 'Що зробити':action, 'Правка':text,
                    'Хотєлка':money, 'Код правки':code, 'Відсоток від':percent,
                    'Ціна правки': price,
                    'На рівні':level, 'За рахунок':at_exp, 
                    'Статус':row['Висновки, обґрунтування'], 'Повний тескт':row['Пропозиції та поправки до проекту']}
    
    return edit_dict, deputies, group, compensated
  return None


In [0]:
# final version
e = pd.DataFrame(columns = ['Номер правки', 'Що зробити', 'Депутат', 
                                'Реєстраційна картка №', 'Правка', 'На рівні',
                                'Відсоток від', 'Ціна правки', 'Сума в бюджеті',
                                'Хотєлка', 'Код правки', 'Категорія', 'За рахунок',
                                'Видалити/коментар', 'Статус', 'Повний тескт']) 
g = e.copy()
c = e.copy()
GDP = 4551.7 * 10**9
# test = edits_df.iloc[202:204]
for index, row in edits_df.iterrows():
  # try:
    edit_parts = split_edit(row['Пропозиції та поправки до проекту'])
    if edit_parts:
      edit_dict = res = None
      raw_number, deputies_list, text = edit_parts
      number = int(get_edit_number(raw_number))
      if number > 742:
        changing = get_sum_changing(text)
        if changing is not 0:
          res = edit_to_dict(raw_number, deputies_list, text, 'замінити', changing)
        # edit_dict, deputies, group, compensated = edit_to_dict(text, 'замінити', changing)  
      action =  get_action(text)
      if action and not res:
        res = edit_to_dict(raw_number, deputies_list, text, action, None)
          # edit_dict, deputies, group, compensated = edit_to_dict(text, action, None)
      if res:
        edit_dict, deputies, group, compensated = res
        for deputy in deputies:
          edit_dict_dep = {**edit_dict, 'Депутат':deputy['deputy'], 
                           'Реєстраційна картка №':deputy['card']}
          if compensated:
            c = c.append(edit_dict_dep, ignore_index=True)
          elif group:
            g = g.append(edit_dict_dep, ignore_index=True)
          else:
            e = e.append(edit_dict_dep, ignore_index=True)
         

In [0]:
# test run
edits = pd.DataFrame(columns = ['Номер правки', 'Що зробити', 'Депутат', 
                                'Реєстраційна картка №', 'Правка', 'На рівні',
                                'Відсоток від', 'Ціна правки', 'Сума в бюджеті',
                                'Хотєлка', 'Код правки', 'Категорія', 'За рахунок',
                                'Видалити/коментар', 'Статус', 'Повний тескт']) 
group_edits = edits.copy()
comp_edits = edits.copy()
GDP = 4551.7 * 10**9
for index, row in edits_df.iterrows():
  # try:
    edit_parts = split_edit(row['Пропозиції та поправки до проекту'])
    if edit_parts:
      raw_number, deputies_list, text = edit_parts
      action =  get_action(text)
      if action:
        # whole_text = text
        action_start = action.start()
        action_end = action.end()
        action = action.group(1)
        # text = text[action.start():]
        at_exp = get_at_expense(text[action_end:])
        compensated = False
        end_lookup = len(text)
        if at_exp:
          end_lookup = action_end + at_exp[0].start()
          # edit_part = text[action_start:action_end + exp_start]
          compensated = at_exp[1]
          at_exp = at_exp[0].group()
        edit_part = text[action_start : end_lookup]
        level = get_level(edit_part)
        price = find_price(edit_part)
        if price:
          money = price
        else:
          money = get_sums(edit_part)
        percent_match = get_percent(edit_part)
        if percent_match:
          percent = percent_match.group()
          if percent_match.group('gdp'):
            percent_number = float(percent_match.group(1).replace(' ','').replace(',','.'))
            money = percent_number*GDP/100
        else: 
          percent = percent_match
        if money or percent or level:
          number = get_edit_number(raw_number)
          deputies, group = get_deputies(deputies_list)
          code = get_code(edit_part)
          for deputy in deputies:
            edit_dict = {'Номер правки':number, 'Що зробити':action, 'Правка':text,
                    'Депутат':deputy['deputy'], 'Реєстраційна картка №':deputy['card'],
                    'Хотєлка':money, 'Код правки':code, 'Відсоток від':percent,
                    'Ціна правки': price,
                    'На рівні':level, 'За рахунок':at_exp, 
                    'Статус':row['Висновки, обґрунтування'], 'Повний тескт':row['Пропозиції та поправки до проекту']}
       
            if compensated:
              comp_edits = comp_edits.append(edit_dict, ignore_index=True)
            elif group:
              group_edits = group_edits.append(edit_dict, ignore_index=True)
            else:
              edits = edits.append(edit_dict, ignore_index=True)
  # except:
  #   print(raw_number, text)
print(len(edits), len(group_edits), len(comp_edits))

In [0]:
with pd.ExcelWriter('/content/drive/My Drive/sums.xlsx') as writer: 
  e.to_excel(writer, sheet_name='Хотєлки', index = False)
  g.to_excel(writer, sheet_name='Групові', index = False)
  c.to_excel(writer, sheet_name='Компенсовані', index = False)
# edits.to_excel('/content/drive/My Drive/budget_test.xlsx', index = False)

In [0]:
!pip install -q xlsxwriter

In [0]:
import xlsxwriter
workbook =xlsxwriter.Workbook('/content/drive/My Drive/sums.xlsx', {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet()
col_num = 0
for key, value in p_edits[2:].items():
    worksheet.write(0, col_num, key)
    worksheet.write_column(1, col_num, value)
    col_num += 1
